## Text Processing Code

In [1]:
import string
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer # BOW
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF
from sklearn.metrics.pairwise import cosine_similarity # Cosine Similarity


### Run code below once to download NLTK resources.

In [2]:
nltk.download('stopwords') 
nltk.download('wordnet')   

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### CODE TINKERING: Bag of Words (BOW)**

In our example, there are only 3 documents in our corpus.

In [3]:
docs = [
    'John has some cats.',
    'Cats, being cats, eat fish.',
    'I ate a big fish.'
]

#### Prepare Stemmer and Stop-Words.

In [4]:
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')

### Perform data cleansing.

In [5]:
def preprocess(docs):
    docs_clean = []
    punc = str.maketrans('', '', string.punctuation)
    for doc in docs:
        doc_no_punc = doc.translate(punc)      # remove punctuations
        words = doc_no_punc.lower().split()    # convert to lowercase and convert into list
        words = [lemmatizer.lemmatize(word, 'v')
                        for word in words if word not in stop_words]    # place word into words list if not already inside
        docs_clean.append(' '.join(words))
    
    return docs_clean

docs_clean = preprocess(docs)
docs_clean #contain sentences that do not contain any punctuations, stopwords

['john cat', 'cat cat eat fish', 'eat big fish']

### Generate our Feature Vectors using Bag of Words.

In [6]:
bow = CountVectorizer()

feature_vectors = bow.fit_transform(docs_clean).toarray() # convert to numpy array
feature_vectors

array([[0, 1, 0, 0, 1],
       [0, 2, 1, 1, 0],
       [1, 0, 1, 1, 0]], dtype=int64)

### View our vocabulary (every unique word in our corpus is a feature)

In [7]:
vocab = bow.get_feature_names()

C:\Users\ASUS\anaconda3\envs\mlexercise\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### Pretty-print our BOW results by combining our vocabulary and feature-vectors into a Pandas' dataframe.

In [8]:
df = pd.DataFrame(data=feature_vectors,
                index=['doc1', 'doc2', 'doc3'],
                columns=vocab)

## CODE TINKERING: TF-IDF**

### Generate feature vectors using TF-IDF.

In [9]:
tfidf = TfidfVectorizer()

#input the preprocessed list of words into the tfidf
feature_vectors = tfidf.fit_transform(docs_clean).toarray()
feature_vectors

array([[0.        , 0.60534851, 0.        , 0.        , 0.79596054],
       [0.        , 0.81649658, 0.40824829, 0.40824829, 0.        ],
       [0.68091856, 0.        , 0.51785612, 0.51785612, 0.        ]])

### View our vocabulary.

In [10]:
vocab = tfidf.get_feature_names()
vocab

['big', 'cat', 'eat', 'fish', 'john']

Pretty-print our TF-IDF results.

In [11]:
df = pd.DataFrame(data=feature_vectors,
                index=['doc1', 'doc2', 'doc3'],
                columns=vocab)

df

,big,cat,eat,fish,john
doc1,0.000000,0.605349,0.000000,0.000000,0.795961
doc2,0.000000,0.816497,0.408248,0.408248,0.000000
doc3,0.680919,0.000000,0.517856,0.517856,0.000000


## CODE TINKERING: Cosine Similarity**

Given a query string, compare it with the corpus for similarity.

In [12]:
query = ['cats and fish']

### Preprocess our query string.

In [13]:
query_clean = preprocess(query)
query_clean

['cat fish']

### Use TF-IDF feature-vectors in our Cosine Similarity computation

In [14]:
# use the same model instance to input
query_feature_vector = tfidf.transform(query_clean).toarray() 
query_feature_vector

array([[0.        , 0.70710678, 0.        , 0.70710678, 0.        ]])

Pretty-print our query's feature vector.

In [15]:
query_df = pd.DataFrame(data=query_feature_vector,
                        index=['query string'],
                        columns=vocab)

query_df

,big,cat,eat,fish,john
query string,0.0,0.707107,0.0,0.707107,0.0


Compute Cosine Similarity between the feature vectors.

In [16]:
similarity = cosine_similarity(query_feature_vector, feature_vectors)

cs = pd.DataFrame(data=similarity,
                index=['cosine similarity'],
                columns=['doc1', 'doc2', 'doc3'])

cs

,doc1,doc2,doc3
cosine similarity,0.428046,0.866025,0.36618
